# Day 11-0: 텍스트 전처리 (Text Preprocessing)

**학습 시간**: 3시간

## 📚 학습 목표

**Part 1: 기초 전처리**
1. **텍스트 정제 기본**: 소문자 변환, 공백 정리, 특수문자 제거
2. **정규표현식 활용**: `re` 모듈로 패턴 기반 텍스트 처리
3. **토큰화 (Tokenization)**: 문장을 단어 단위로 분리
4. **불용어 제거**: NLTK stopwords로 의미 없는 단어 제거
5. **어간 추출 (Stemming)**: 단어의 어근 추출

**Part 2: 한글 처리**
1. **KoNLPy 소개**: 한국어 자연어 처리 라이브러리
2. **형태소 분석**: Okt, Komoran으로 형태소 분리
3. **품사 태깅 (POS Tagging)**: 명사, 동사, 형용사 구분
4. **명사 추출**: 핵심 키워드 추출 기법

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 텍스트 정제 | 데이터 품질 향상 | 뉴스 기사에서 HTML 태그 제거 |
| 토큰화 | 분석 단위 생성 | 고객 리뷰를 단어별로 분리 |
| 불용어 제거 | 노이즈 감소 | "은", "는", "이", "가" 제거 |
| 형태소 분석 | 한글 처리 | "먹었습니다" → "먹다" 추출 |
| 명사 추출 | 키워드 분석 | 리뷰에서 "배송", "품질" 추출 |

**분석가 관점**: 텍스트 전처리는 모든 NLP 파이프라인의 첫 단계입니다. 깨끗한 데이터가 좋은 분석 결과를 만듭니다!

---

## 라이브러리 설치 및 임포트

텍스트 전처리에 필요한 라이브러리들을 설치하고 임포트합니다.

In [1]:
!uv add nltk konlpy

Resolved 141 packages in 801ms                                       
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)-------------------     0 B/568.73 KiB          
⠙ Preparing packages... (0/3)------------------- 16.00 KiB/568.73 KiB        
⠙ Preparing packages... (0/3)------------------- 16.00 KiB/568.73 KiB        
regex                ------------------------------     0 B/282.12 KiB
⠙ Preparing packages... (0/3)------------------- 16.00 KiB/568.73 KiB        
regex                ------------------------------ 14.89 KiB/282.12 KiB
⠙ Preparing packages... (0/3)------------------- 16.00 KiB/568.73 KiB        
regex                ------------------------------ 14.89 KiB/282.12 KiB
⠙ Preparing packages... (0/3)------------------- 32.00 KiB/568.73 KiB        
regex                ------------------------------ 14.89 KiB/282.12 KiB
⠙ Preparing packages... (0/3)------------------- 48.00 KiB/568.73 KiB        
regex        

In [2]:
# 필요한 라이브러리 설치 (최초 1회)
# !pip install nltk konlpy

# NLTK 데이터 다운로드 (최초 1회)
import nltk
nltk.download('punkt')        # 토큰화
nltk.download('stopwords')    # 불용어
nltk.download('wordnet')      # 표제어 추출
nltk.download('punkt_tab')    # 토큰화 추가 데이터

[nltk_data] Downloading package punkt to /Users/dante/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/dante/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/dante/nltk_data...
[nltk_data] Downloading package punkt_tab to /Users/dante/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
!ls -al ~/nltk*

total 16
drwxr-xr-x@   6 dante  staff   192 Jul 11  2024 .
drwxr-x---+ 231 dante  staff  7392 Jan 15 13:27 ..
-rw-r--r--@   1 dante  staff  6148 Nov 18 16:59 .DS_Store
drwxr-xr-x@   5 dante  staff   160 Jan 15 13:48 corpora
drwxr-xr-x@   7 dante  staff   224 Nov 18 16:58 taggers
drwxr-xr-x@   7 dante  staff   224 Nov 22 12:40 tokenizers


In [4]:
# 기본 라이브러리
import re
from collections import Counter

# NLTK (영어 처리)
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# KoNLPy (한글 처리)
from konlpy.tag import Okt, Komoran

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


---

# Part 1: 기초 전처리

---

## 1.1 텍스트 정제 기본

텍스트 분석의 첫 단계는 **데이터 정제**입니다. 노이즈를 제거하고 일관된 형식으로 변환합니다.

### 기본 정제 기법
1. **소문자 변환**: 대소문자 통일
2. **공백 정리**: 중복 공백 제거
3. **앞뒤 공백 제거**: strip()

In [5]:
# [예제 1] 기본 텍스트 정제
raw_text = "  Hello  WORLD!   Welcome to   TEXT  Processing.  "

# 1. 소문자 변환
lower_text = raw_text.lower()
print(f"소문자: '{lower_text}'")

# 2. 앞뒤 공백 제거
stripped = lower_text.strip()
print(f"strip(): '{stripped}'")

# 3. 중복 공백을 단일 공백으로 (정규표현식)
cleaned = re.sub(r'\s+', ' ', stripped)
print(f"최종: '{cleaned}'")

소문자: '  hello  world!   welcome to   text  processing.  '
strip(): 'hello  world!   welcome to   text  processing.'
최종: 'hello world! welcome to text processing.'


In [6]:
# 💡 실무 예시: 고객 리뷰 정제
customer_reviews = [
    "  배송이 너무 빨라요!!!  ",
    "품질   좋습니다~~~ 추천해요  ",
    "   가격대비 BEST 상품입니다.   "
]

def clean_basic(text):
    """기본 텍스트 정제 함수"""
    text = text.lower()              # 소문자
    text = text.strip()              # 앞뒤 공백
    text = re.sub(r'\s+', ' ', text) # 중복 공백
    return text

cleaned_reviews = [clean_basic(review) for review in customer_reviews]
for original, cleaned in zip(customer_reviews, cleaned_reviews):
    print(f"원본: '{original}'")
    print(f"정제: '{cleaned}'")
    print()

원본: '  배송이 너무 빨라요!!!  '
정제: '배송이 너무 빨라요!!!'

원본: '품질   좋습니다~~~ 추천해요  '
정제: '품질 좋습니다~~~ 추천해요'

원본: '   가격대비 BEST 상품입니다.   '
정제: '가격대비 best 상품입니다.'



---

## 1.2 정규표현식 (Regular Expression)

**정규표현식**은 텍스트에서 패턴을 찾고 치환하는 강력한 도구입니다.

### 주요 메타문자
| 패턴 | 의미 | 예시 |
|------|------|------|
| `\d` | 숫자 | `[0-9]` |
| `\w` | 단어 문자 | `[a-zA-Z0-9_]` |
| `\s` | 공백 문자 | 스페이스, 탭, 줄바꿈 |
| `.` | 모든 문자 | 줄바꿈 제외 |
| `+` | 1개 이상 | `a+` → "a", "aa", "aaa" |
| `*` | 0개 이상 | `a*` → "", "a", "aa" |
| `[]` | 문자 클래스 | `[abc]` → a, b, c 중 하나 |

In [7]:
# [예제 2] 특수문자 제거
text = "Hello! How are you? I'm fine, thank you~ :)"

# 방법 1: 특정 문자만 제거
clean1 = re.sub(r'[!?~:()]', '', text)
print(f"방법 1: {clean1}")


방법 1: Hello How are you I'm fine, thank you 


In [8]:

# 방법 2: 알파벳과 공백만 남기기
clean2 = re.sub(r'[^a-zA-Z\s]', '', text)
print(f"방법 2: {clean2}")


방법 2: Hello How are you Im fine thank you 


In [9]:

# 방법 3: 알파벳, 숫자, 공백만 남기기
text_with_num = "Price: $100, Discount: 20%!"
clean3 = re.sub(r'[^a-zA-Z0-9\s]', '', text_with_num)
print(f"방법 3: {clean3}")

방법 3: Price 100 Discount 20


In [10]:
# [예제 3] 한글 텍스트에서 특수문자 제거
korean_text = "배송이 빠르네요!!! 최고에요~~~ ^^ 별점: 5점"

# 한글, 영어, 숫자, 공백만 남기기
clean_korean = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', korean_text)
print(f"원본: {korean_text}")
print(f"정제: {clean_korean}")

원본: 배송이 빠르네요!!! 최고에요~~~ ^^ 별점: 5점
정제: 배송이 빠르네요 최고에요  별점 5점


In [11]:
# 💡 실무 예시: 다양한 정규표현식 패턴

# 1. 이메일 추출
text_email = "문의: support@company.com 또는 sales@shop.co.kr"
emails = re.findall(r'[\w.-]+@[\w.-]+', text_email)
print(f"이메일: {emails}")


이메일: ['support@company.com', 'sales@shop.co.kr']


In [12]:

# 2. 전화번호 추출
text_phone = "연락처: 010-1234-5678, 02-123-4567"
phones = re.findall(r'\d{2,3}-\d{3,4}-\d{4}', text_phone)
print(f"전화번호: {phones}")


전화번호: ['010-1234-5678', '02-123-4567']


In [13]:

# 3. 해시태그 추출
text_hashtag = "오늘 날씨 좋다 #서울 #맑음 #산책 #주말"
hashtags = re.findall(r'#\w+', text_hashtag)
print(f"해시태그: {hashtags}")

해시태그: ['#서울', '#맑음', '#산책', '#주말']


In [14]:

# 4. URL 제거
text_url = "자세한 내용은 https://example.com/page 참고하세요"
clean_url = re.sub(r'https?://\S+', '', text_url)
print(f"URL 제거: {clean_url}")

URL 제거: 자세한 내용은  참고하세요


In [15]:
# [예제 4] HTML 태그 제거
html_text = "<p>안녕하세요!</p><br><strong>중요한 공지</strong>입니다."

# HTML 태그 제거
clean_html = re.sub(r'<[^>]+>', '', html_text)
print(f"원본: {html_text}")
print(f"정제: {clean_html}")

원본: <p>안녕하세요!</p><br><strong>중요한 공지</strong>입니다.
정제: 안녕하세요!중요한 공지입니다.


---

## 1.3 토큰화 (Tokenization)

**토큰화**는 텍스트를 분석 단위(토큰)로 분리하는 과정입니다.

### 토큰화 종류
- **단어 토큰화 (Word Tokenization)**: 단어 단위로 분리
- **문장 토큰화 (Sentence Tokenization)**: 문장 단위로 분리

In [16]:
# [예제 5] 기본 토큰화 (split)
sentence = "I love natural language processing"

# 공백 기준 분리
tokens = sentence.split()
print(f"split(): {tokens}")

split(): ['I', 'love', 'natural', 'language', 'processing']


In [17]:
# [예제 6] NLTK 토큰화 (더 정교함)
text = "Hello! I'm learning NLP. It's amazing, isn't it?"

# 단어 토큰화
word_tokens = word_tokenize(text)
print(f"단어 토큰: {word_tokens}")


단어 토큰: ['Hello', '!', 'I', "'m", 'learning', 'NLP', '.', 'It', "'s", 'amazing', ',', 'is', "n't", 'it', '?']


In [18]:

# 문장 토큰화
sent_tokens = sent_tokenize(text)
print(f"문장 토큰: {sent_tokens}")

문장 토큰: ['Hello!', "I'm learning NLP.", "It's amazing, isn't it?"]


In [19]:
# 💡 실무 예시: 뉴스 기사 토큰화
news_article = """
서울시는 오늘 새로운 교통 정책을 발표했다. 
이 정책은 대중교통 이용을 촉진하기 위한 것이다.
시민들의 반응은 대체로 긍정적이다.
"""

# 문장 분리
sentences = sent_tokenize(news_article.strip())
print(f"총 {len(sentences)}개 문장:")
for i, sent in enumerate(sentences, 1):
    print(f"  {i}. {sent}")

총 3개 문장:
  1. 서울시는 오늘 새로운 교통 정책을 발표했다.
  2. 이 정책은 대중교통 이용을 촉진하기 위한 것이다.
  3. 시민들의 반응은 대체로 긍정적이다.


---

## 1.4 불용어 제거 (Stopword Removal)

**불용어(Stopwords)**는 분석에 큰 의미가 없는 단어들입니다.
- 영어: "the", "is", "at", "which", "on" 등
- 한글: "은", "는", "이", "가", "을", "를" 등

In [20]:
# [예제 7] 영어 불용어 제거
from nltk.corpus import stopwords

# 영어 불용어 목록
english_stopwords = set(stopwords.words('english'))
print(f"영어 불용어 수: {len(english_stopwords)}")
print(f"예시: {list(english_stopwords)[:10]}")

영어 불용어 수: 198
예시: ['and', "hadn't", 'same', 'myself', 'did', 'just', 'the', 'before', 'once', "they'd"]


In [21]:
# [예제 8] 불용어 제거 적용
text = "This is an example showing how to remove stopwords from text"

# 토큰화
tokens = word_tokenize(text.lower())
print(f"원본 토큰: {tokens}")

# 불용어 제거
filtered_tokens = [token for token in tokens if token not in english_stopwords]
print(f"불용어 제거: {filtered_tokens}")

원본 토큰: ['this', 'is', 'an', 'example', 'showing', 'how', 'to', 'remove', 'stopwords', 'from', 'text']
불용어 제거: ['example', 'showing', 'remove', 'stopwords', 'text']


In [22]:
# 💡 실무 예시: 커스텀 불용어 추가
text = "The product is very good and the delivery was fast"

# 기본 불용어 + 커스텀 불용어
custom_stopwords = english_stopwords.union({'very', 'really', 'just'})

tokens = word_tokenize(text.lower())
filtered = [t for t in tokens if t not in custom_stopwords and t.isalpha()]
print(f"원본: {tokens}")
print(f"필터: {filtered}")

원본: ['the', 'product', 'is', 'very', 'good', 'and', 'the', 'delivery', 'was', 'fast']
필터: ['product', 'good', 'delivery', 'fast']


---

## 1.5 어간 추출 (Stemming) vs 표제어 추출 (Lemmatization)

### 어간 추출 (Stemming)
- 규칙 기반으로 단어의 **어미를 제거**
- 빠르지만 정확도 낮음
- 예: "running" → "run", "studies" → "studi"

### 표제어 추출 (Lemmatization)
- 사전을 이용해 **기본형(lemma)** 찾기
- 느리지만 정확도 높음
- 예: "running" → "run", "better" → "good"

In [23]:
# [예제 9] 어간 추출 (Stemming)
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

words = ['running', 'runs', 'ran', 'easily', 'fairly', 'studies', 'studying']

print("어간 추출 결과:")
for word in words:
    stemmed = stemmer.stem(word)
    print(f"  {word:12} → {stemmed}")

어간 추출 결과:
  running      → run
  runs         → run
  ran          → ran
  easily       → easili
  fairly       → fairli
  studies      → studi
  studying     → studi


In [24]:
# [예제 10] 표제어 추출 (Lemmatization)
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = ['running', 'runs', 'ran', 'better', 'studies', 'studying']

print("표제어 추출 결과:")
for word in words:
    # 동사(v)로 처리
    lemma_v = lemmatizer.lemmatize(word, pos='v')
    # 명사(n)로 처리
    lemma_n = lemmatizer.lemmatize(word, pos='n')
    print(f"  {word:12} → 동사: {lemma_v:10} 명사: {lemma_n}")

표제어 추출 결과:
  running      → 동사: run        명사: running
  runs         → 동사: run        명사: run
  ran          → 동사: run        명사: ran
  better       → 동사: better     명사: better
  studies      → 동사: study      명사: study
  studying     → 동사: study      명사: studying


In [25]:
# 💡 실무 예시: 어간 추출 vs 표제어 추출 비교
sentence = "The studies showed that cats are running faster than dogs"
tokens = word_tokenize(sentence.lower())

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

stemmed = [stemmer.stem(t) for t in tokens]
lemmatized = [lemmatizer.lemmatize(t, pos='v') for t in tokens]

print(f"원본:     {tokens}")
print(f"Stemmed:  {stemmed}")
print(f"Lemma:    {lemmatized}")

원본:     ['the', 'studies', 'showed', 'that', 'cats', 'are', 'running', 'faster', 'than', 'dogs']
Stemmed:  ['the', 'studi', 'show', 'that', 'cat', 'are', 'run', 'faster', 'than', 'dog']
Lemma:    ['the', 'study', 'show', 'that', 'cat', 'be', 'run', 'faster', 'than', 'dog']


---

# Part 2: 한글 처리 (KoNLPy)

---

## 2.1 KoNLPy 소개

**KoNLPy**는 한국어 자연어 처리를 위한 파이썬 라이브러리입니다.

### 형태소 분석기 종류
| 분석기 | 특징 | 속도 | 정확도 |
|--------|------|------|--------|
| **Okt** | Twitter 기반, 간편함 | 빠름 | 보통 |
| **Komoran** | Java 기반, 사용자 사전 | 보통 | 높음 |
| **Mecab** | C 기반, 가장 빠름 | 매우 빠름 | 높음 |
| **Kkma** | 정확하지만 느림 | 느림 | 매우 높음 |
| **Hannanum** | KAIST 개발 | 보통 | 보통 |

In [26]:
# [예제 11] Okt 형태소 분석기
from konlpy.tag import Okt

okt = Okt()

text = "아버지가방에들어가신다"

# 형태소 분석
morphs = okt.morphs(text)
print(f"형태소: {morphs}")


형태소: ['아버지', '가방', '에', '들어가신다']


In [27]:

# 품사 태깅
pos_tags = okt.pos(text)
print(f"품사 태깅: {pos_tags}")


품사 태깅: [('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가신다', 'Verb')]


In [28]:

# 명사 추출
nouns = okt.nouns(text)
print(f"명사: {nouns}")

명사: ['아버지', '가방']


In [29]:
# [예제 12] Komoran 형태소 분석기
from konlpy.tag import Komoran

komoran = Komoran()

text = "코로나19로 인해 재택근무가 늘어났습니다"

# 형태소 분석
morphs = komoran.morphs(text)
print(f"형태소: {morphs}")

# 품사 태깅
pos_tags = komoran.pos(text)
print(f"품사 태깅: {pos_tags}")

# 명사 추출
nouns = komoran.nouns(text)
print(f"명사: {nouns}")

형태소: ['코로나', '19', '로', '인해', '재택근무', '가', '늘어나', '았', '습니다']
품사 태깅: [('코로나', 'NNP'), ('19', 'NNP'), ('로', 'JKB'), ('인해', 'NNP'), ('재택근무', 'NNP'), ('가', 'JKS'), ('늘어나', 'VV'), ('았', 'EP'), ('습니다', 'EC')]
명사: ['코로나', '19', '인해', '재택근무']


---

## 2.2 형태소 분석 (Morphological Analysis)

**형태소**는 의미를 가진 가장 작은 언어 단위입니다.
- "먹었습니다" → "먹" + "었" + "습니다"
- "자연어처리" → "자연어" + "처리"

In [30]:
# [예제 13] 형태소 분석 상세
okt = Okt()

sentences = [
    "맛있는 음식을 먹었습니다",
    "서울에서 부산까지 KTX로 2시간 걸립니다",
    "오늘 날씨가 정말 좋네요"
]

for sent in sentences:
    print(f"\n원문: {sent}")
    print(f"형태소: {okt.morphs(sent)}")
    print(f"품사: {okt.pos(sent)}")


원문: 맛있는 음식을 먹었습니다
형태소: ['맛있는', '음식', '을', '먹었습니다']
품사: [('맛있는', 'Adjective'), ('음식', 'Noun'), ('을', 'Josa'), ('먹었습니다', 'Verb')]

원문: 서울에서 부산까지 KTX로 2시간 걸립니다
형태소: ['서울', '에서', '부산', '까지', 'KTX', '로', '2시간', '걸립니다']
품사: [('서울', 'Noun'), ('에서', 'Josa'), ('부산', 'Noun'), ('까지', 'Josa'), ('KTX', 'Alpha'), ('로', 'Noun'), ('2시간', 'Number'), ('걸립니다', 'Verb')]

원문: 오늘 날씨가 정말 좋네요
형태소: ['오늘', '날씨', '가', '정말', '좋네요']
품사: [('오늘', 'Noun'), ('날씨', 'Noun'), ('가', 'Josa'), ('정말', 'Noun'), ('좋네요', 'Adjective')]


In [31]:
# 💡 실무 예시: 고객 리뷰 형태소 분석
reviews = [
    "배송이 빠르고 상품 품질이 좋아요",
    "가격은 비싸지만 성능이 뛰어납니다",
    "포장이 꼼꼼하게 잘 되어있었어요"
]

okt = Okt()

for review in reviews:
    # 형태소 분석 (정규화 + 어간 추출)
    morphs = okt.morphs(review, norm=True, stem=True)
    print(f"원문: {review}")
    print(f"분석: {morphs}")
    print()

원문: 배송이 빠르고 상품 품질이 좋아요
분석: ['배송', '이', '빠르다', '상품', '품질', '이', '좋다']

원문: 가격은 비싸지만 성능이 뛰어납니다
분석: ['가격', '은', '비싸다', '성능', '이', '뛰어나다']

원문: 포장이 꼼꼼하게 잘 되어있었어요
분석: ['포장', '이', '꼼꼼하다', '자다', '되어다']



---

## 2.3 품사 태깅 (POS Tagging)

**품사 태깅**은 각 형태소에 품사 정보를 부여하는 과정입니다.

### Okt 주요 품사 태그
| 태그 | 품사 | 예시 |
|------|------|------|
| Noun | 명사 | 사람, 컴퓨터 |
| Verb | 동사 | 가다, 먹다 |
| Adjective | 형용사 | 좋다, 크다 |
| Adverb | 부사 | 매우, 빨리 |
| Josa | 조사 | 은, 는, 이, 가 |
| Eomi | 어미 | ~다, ~요 |
| Punctuation | 구두점 | ., !, ? |

In [32]:
# [예제 14] 품사별 단어 추출
okt = Okt()

text = "서울은 대한민국의 수도이며 매우 아름다운 도시입니다"

# 전체 품사 태깅
pos_result = okt.pos(text)
print(f"전체: {pos_result}")

# 품사별 분류
nouns = [word for word, pos in pos_result if pos == 'Noun']
verbs = [word for word, pos in pos_result if pos == 'Verb']
adjectives = [word for word, pos in pos_result if pos == 'Adjective']
adverbs = [word for word, pos in pos_result if pos == 'Adverb']

print(f"\n명사: {nouns}")
print(f"동사: {verbs}")
print(f"형용사: {adjectives}")
print(f"부사: {adverbs}")

전체: [('서울', 'Noun'), ('은', 'Josa'), ('대한민국', 'Noun'), ('의', 'Josa'), ('수도', 'Noun'), ('이며', 'Josa'), ('매우', 'Noun'), ('아름다운', 'Adjective'), ('도시', 'Noun'), ('입니다', 'Adjective')]

명사: ['서울', '대한민국', '수도', '매우', '도시']
동사: []
형용사: ['아름다운', '입니다']
부사: []


In [33]:
# 💡 실무 예시: 감성 분석용 형용사 추출
reviews = [
    "이 제품은 정말 좋고 편리해요",
    "배송은 빠르지만 품질이 별로예요",
    "가격이 저렴하고 만족스럽습니다"
]

okt = Okt()

for review in reviews:
    pos_result = okt.pos(review, stem=True)  # 어간 추출 적용
    
    adjectives = [word for word, pos in pos_result if pos == 'Adjective']
    
    print(f"리뷰: {review}")
    print(f"형용사: {adjectives}")
    print()

리뷰: 이 제품은 정말 좋고 편리해요
형용사: ['좋다', '편리하다']

리뷰: 배송은 빠르지만 품질이 별로예요
형용사: ['빠르다']

리뷰: 가격이 저렴하고 만족스럽습니다
형용사: ['저렴하다', '만족스럽다']



---

## 2.4 명사 추출

**명사 추출**은 텍스트에서 핵심 키워드를 찾는 가장 기본적인 방법입니다.

In [34]:
# [예제 15] 명사 추출 및 빈도 분석
from collections import Counter

okt = Okt()

news = """
삼성전자가 새로운 스마트폰을 출시했다. 
이번 스마트폰은 카메라 성능이 크게 향상되었다.
삼성전자 관계자는 이번 제품이 시장에서 큰 반응을 얻을 것으로 기대한다고 밝혔다.
애플과의 경쟁에서도 우위를 점할 것으로 예상된다.
"""

# 명사 추출
nouns = okt.nouns(news)
print(f"추출된 명사: {nouns}")

# 빈도 분석
noun_counts = Counter(nouns)
print(f"\n상위 5개 명사:")
for noun, count in noun_counts.most_common(5):
    print(f"  {noun}: {count}회")

추출된 명사: ['전자', '스마트폰', '출시', '이번', '스마트폰', '카메라', '성능', '크게', '향상', '삼성', '전자', '관계자', '이번', '제품', '시장', '반응', '것', '애플', '경쟁', '우위', '점', '것', '예상']

상위 5개 명사:
  전자: 2회
  스마트폰: 2회
  이번: 2회
  것: 2회
  출시: 1회


In [35]:
# 💡 실무 예시: SNS 댓글 키워드 분석
comments = [
    "이 영화 진짜 재밌어요 배우 연기 최고",
    "스토리가 탄탄하고 영상미가 뛰어남",
    "배우들 연기력이 대단해요 감동 받음",
    "영화 OST도 좋고 스토리도 감동적",
    "이번 주말에 가족이랑 영화 보러 가야겠어요"
]

okt = Okt()

# 모든 댓글에서 명사 추출
all_nouns = []
for comment in comments:
    nouns = okt.nouns(comment)
    all_nouns.extend(nouns)

# 1글자 명사 제외 (노이즈 제거)
filtered_nouns = [n for n in all_nouns if len(n) > 1]

# 키워드 빈도 분석
keyword_counts = Counter(filtered_nouns)
print("주요 키워드:")
for keyword, count in keyword_counts.most_common(10):
    print(f"  {keyword}: {count}회")

주요 키워드:
  영화: 3회
  배우: 2회
  스토리: 2회
  감동: 2회
  진짜: 1회
  연기: 1회
  최고: 1회
  영상: 1회
  미가: 1회
  연기력: 1회


---

## 2.5 종합: 텍스트 전처리 파이프라인

실무에서는 여러 전처리 기법을 순차적으로 적용합니다.

In [36]:
# [예제 16] 한글 텍스트 전처리 파이프라인
def preprocess_korean(text, min_length=2):
    """
    한글 텍스트 전처리 파이프라인
    
    Args:
        text (str): 원본 텍스트
        min_length (int): 최소 단어 길이
    
    Returns:
        list: 전처리된 토큰 리스트
    """
    okt = Okt()
    
    # 1. 특수문자 제거 (한글, 영어, 숫자, 공백만 유지)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    
    # 2. 중복 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 3. 형태소 분석 (정규화 + 어간 추출)
    tokens = okt.morphs(text, norm=True, stem=True)
    
    # 4. 불용어 제거 (한글 조사, 어미 등)
    korean_stopwords = {'은', '는', '이', '가', '을', '를', '의', '에', '에서', 
                        '로', '으로', '도', '만', '과', '와', '하다', '있다', '되다'}
    tokens = [t for t in tokens if t not in korean_stopwords]
    
    # 5. 최소 길이 필터링
    tokens = [t for t in tokens if len(t) >= min_length]
    
    return tokens

# 테스트
sample_text = "오늘 배송받은 상품이 정말 좋아요!!! 품질도 최고입니다~~ 강력 추천해요 ^^;;;"
result = preprocess_korean(sample_text)

print(f"원본: {sample_text}")
print(f"결과: {result}")

원본: 오늘 배송받은 상품이 정말 좋아요!!! 품질도 최고입니다~~ 강력 추천해요 ^^;;;
결과: ['오늘', '배송', '받다', '상품', '정말', '좋다', '품질', '최고', '이다', '강력', '추천']


In [37]:
# [예제 17] 영어 텍스트 전처리 파이프라인
def preprocess_english(text, min_length=2):
    """
    영어 텍스트 전처리 파이프라인
    
    Args:
        text (str): 원본 텍스트
        min_length (int): 최소 단어 길이
    
    Returns:
        list: 전처리된 토큰 리스트
    """
    # 1. 소문자 변환
    text = text.lower()
    
    # 2. 특수문자 제거
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. 중복 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 4. 토큰화
    tokens = word_tokenize(text)
    
    # 5. 불용어 제거
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words]
    
    # 6. 표제어 추출
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t, pos='v') for t in tokens]
    
    # 7. 최소 길이 필터링
    tokens = [t for t in tokens if len(t) >= min_length]
    
    return tokens

# 테스트
sample_text = "I was running in the park yesterday! The weather was amazing."
result = preprocess_english(sample_text)

print(f"원본: {sample_text}")
print(f"결과: {result}")

원본: I was running in the park yesterday! The weather was amazing.
결과: ['run', 'park', 'yesterday', 'weather', 'amaze']


---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 기본 텍스트 정제 ⭐

**문제**: 아래 텍스트를 정제하는 함수를 작성하세요.

**요구사항**:
- 소문자 변환
- 앞뒤 공백 제거
- 중복 공백을 단일 공백으로 변환

```python
raw_text = "  Hello   WORLD!  Welcome to   NLP.  "
```

**기대 결과**: `"hello world! welcome to nlp."`

In [ ]:
# 여기에 코드를 작성하세요
raw_text = "  Hello   WORLD!  Welcome to   NLP.  "


### Q2. 정규표현식 - 특수문자 제거 ⭐

**문제**: 정규표현식을 사용하여 특수문자를 제거하세요.

**요구사항**:
- 한글, 영어, 숫자, 공백만 남기기
- 나머지 문자는 모두 제거

```python
text = "배송 빠르네요!!! 가격 10,000원 @#$% 추천해요~~"
```

**기대 결과**: `"배송 빠르네요 가격 10000원  추천해요"`

In [ ]:
# 여기에 코드를 작성하세요
text = "배송 빠르네요!!! 가격 10,000원 @#$% 추천해요~~"


### Q3. 정규표현식 - 해시태그 추출 ⭐⭐

**문제**: SNS 게시물에서 모든 해시태그를 추출하세요.

**요구사항**:
- `re.findall()` 사용
- 해시태그 형식: `#단어`

```python
post = "오늘 날씨 좋다 #서울 #맑음 #주말나들이 점심은 #맛집 가야지"
```

**기대 결과**: `['#서울', '#맑음', '#주말나들이', '#맛집']`

In [ ]:
# 여기에 코드를 작성하세요
post = "오늘 날씨 좋다 #서울 #맑음 #주말나들이 점심은 #맛집 가야지"


### Q4. NLTK 토큰화 ⭐⭐

**문제**: 영어 문장을 단어와 문장으로 토큰화하세요.

**요구사항**:
- `word_tokenize()`: 단어 토큰화
- `sent_tokenize()`: 문장 토큰화

```python
text = "Hello! How are you? I'm learning NLP. It's really fun."
```

**기대 결과**:
- 단어: `['Hello', '!', 'How', 'are', 'you', '?', ...]`
- 문장: `['Hello!', 'How are you?', ...]`

In [ ]:
# 여기에 코드를 작성하세요
text = "Hello! How are you? I'm learning NLP. It's really fun."


### Q5. 불용어 제거 ⭐⭐⭐

**문제**: 영어 문장에서 불용어를 제거하세요.

**요구사항**:
- 소문자 변환 후 토큰화
- NLTK 불용어 제거
- 알파벳으로만 이루어진 단어만 유지

```python
sentence = "The movie was really amazing and I loved it very much!"
```

**기대 결과**: `['movie', 'really', 'amazing', 'loved', 'much']`

In [ ]:
# 여기에 코드를 작성하세요
sentence = "The movie was really amazing and I loved it very much!"


### Q6. 어간 추출 vs 표제어 추출 ⭐⭐⭐

**문제**: 단어 리스트에 어간 추출과 표제어 추출을 각각 적용하세요.

**요구사항**:
- `PorterStemmer` 사용
- `WordNetLemmatizer` 사용 (동사 품사)
- 결과를 딕셔너리로 반환: `{원본: (stemmed, lemmatized)}`

```python
words = ['running', 'studies', 'played', 'better', 'happily']
```

**기대 결과**: 
```
running: stem=run, lemma=run
studies: stem=studi, lemma=study
...
```

In [ ]:
# 여기에 코드를 작성하세요
words = ['running', 'studies', 'played', 'better', 'happily']


### Q7. KoNLPy 명사 추출 ⭐⭐⭐

**문제**: 고객 리뷰에서 명사를 추출하고 빈도를 분석하세요.

**요구사항**:
- `Okt.nouns()` 사용
- 1글자 명사 제외
- 상위 5개 명사 출력

```python
reviews = [
    "배송이 빠르고 상품 품질이 좋아요",
    "가격대비 품질이 훌륭합니다",
    "배송은 빠른데 포장이 아쉬워요",
    "상품 퀄리티 최고 배송도 빠름"
]
```

**기대 결과**:
```
배송: 3회
품질: 2회
상품: 2회
...
```

In [ ]:
# 여기에 코드를 작성하세요
reviews = [
    "배송이 빠르고 상품 품질이 좋아요",
    "가격대비 품질이 훌륭합니다",
    "배송은 빠른데 포장이 아쉬워요",
    "상품 퀄리티 최고 배송도 빠름"
]


### Q8. 품사별 단어 분류 ⭐⭐⭐⭐

**문제**: 문장에서 품사별로 단어를 분류하세요.

**요구사항**:
- `Okt.pos()` 사용
- 명사(Noun), 동사(Verb), 형용사(Adjective) 분류
- 결과를 딕셔너리로 반환

```python
text = "맛있는 음식을 먹고 행복한 하루를 보냈다"
```

**기대 결과**:
```python
{
    'Noun': ['음식', '하루'],
    'Verb': ['먹다', '보내다'],
    'Adjective': ['맛있다', '행복하다']
}
```

In [ ]:
# 여기에 코드를 작성하세요
text = "맛있는 음식을 먹고 행복한 하루를 보냈다"


### Q9. 한글 전처리 함수 ⭐⭐⭐⭐

**문제**: 한글 텍스트 전처리 함수를 작성하세요.

**요구사항**:
- 함수명: `clean_korean_text`
- 기능:
  1. 특수문자 제거 (한글, 영어, 숫자, 공백만 유지)
  2. 중복 공백 제거
  3. 형태소 분석 (정규화, 어간 추출)
  4. 불용어 제거 (조사, 어미)
  5. 2글자 이상 토큰만 유지
- 반환: 토큰 리스트

```python
text = "오늘 배송받은 상품이 정말 좋아요!!! 강력 추천합니다~~"
```

**기대 결과**: `['오늘', '배송', '받다', '상품', '정말', '좋다', '강력', '추천']` (유사한 결과)

In [ ]:
# 여기에 코드를 작성하세요
text = "오늘 배송받은 상품이 정말 좋아요!!! 강력 추천합니다~~"


### Q10. 종합: 뉴스 기사 키워드 추출 ⭐⭐⭐⭐⭐

**문제**: 뉴스 기사에서 핵심 키워드를 추출하는 파이프라인을 완성하세요.

**요구사항**:
- 함수명: `extract_keywords`
- 입력: 뉴스 기사 텍스트
- 처리 과정:
  1. 특수문자 제거
  2. 형태소 분석 (Okt)
  3. 명사만 추출
  4. 1글자 명사 제외
  5. 빈도 분석
- 반환: 상위 N개 키워드 리스트 `[(키워드, 빈도), ...]`

```python
news = """
삼성전자가 새로운 갤럭시 스마트폰을 출시했다.
이번 갤럭시는 카메라 성능이 크게 향상되었으며,
배터리 용량도 늘어났다. 삼성전자 관계자는
"이번 갤럭시가 시장에서 좋은 반응을 얻을 것"이라고 밝혔다.
경쟁사인 애플의 아이폰과의 경쟁도 치열해질 전망이다.
"""
```

**기대 결과**: `[('갤럭시', 3), ('삼성전자', 2), ('스마트폰', 1), ...]`

In [ ]:
# 여기에 코드를 작성하세요
news = """
삼성전자가 새로운 갤럭시 스마트폰을 출시했다.
이번 갤럭시는 카메라 성능이 크게 향상되었으며,
배터리 용량도 늘어났다. 삼성전자 관계자는
"이번 갤럭시가 시장에서 좋은 반응을 얻을 것"이라고 밝혔다.
경쟁사인 애플의 아이폰과의 경쟁도 치열해질 전망이다.
"""


---

## 📊 학습 정리

### Part 1: 기초 전처리 핵심 요약

| 기법 | 목적 | 주요 함수/패턴 |
|------|------|---------------|
| 소문자 변환 | 대소문자 통일 | `text.lower()` |
| 공백 정리 | 노이즈 제거 | `strip()`, `re.sub(r'\s+', ' ')` |
| 특수문자 제거 | 패턴 기반 정제 | `re.sub(r'[^가-힣a-zA-Z0-9\s]', '')` |
| 토큰화 | 분석 단위 분리 | `word_tokenize()`, `sent_tokenize()` |
| 불용어 제거 | 의미 없는 단어 제거 | `stopwords.words('english')` |
| 어간 추출 | 단어 정규화 | `PorterStemmer()` |
| 표제어 추출 | 사전 기반 정규화 | `WordNetLemmatizer()` |

### Part 2: 한글 처리 핵심 요약

| 기법 | 목적 | 주요 함수 |
|------|------|----------|
| 형태소 분석 | 형태소 분리 | `okt.morphs()` |
| 품사 태깅 | 품사 부여 | `okt.pos()` |
| 명사 추출 | 키워드 추출 | `okt.nouns()` |
| 정규화 | 비표준 표현 정제 | `okt.morphs(norm=True)` |
| 어간 추출 | 원형 복원 | `okt.morphs(stem=True)` |

### 💡 실무 팁

1. **전처리 순서가 중요**: 특수문자 제거 → 토큰화 → 불용어 제거 → 정규화
2. **한글은 형태소 분석이 필수**: 띄어쓰기만으로는 의미 단위 분리 불가
3. **커스텀 불용어 목록 관리**: 도메인별로 추가/제거할 단어가 다름
4. **정규화 vs 어간 추출**: 정확도 vs 속도 트레이드오프 고려
5. **분석 목적에 따른 품사 선택**: 감성 분석은 형용사, 키워드 분석은 명사

---

## 🚀 다음 단계

텍스트 전처리를 마스터했습니다! 다음은 **텍스트 분석 기법**입니다.

**다음 노트북**: `Day11_1_텍스트_분석_기법_Text_Analysis.ipynb`
- 단어 빈도 분석 (Word Frequency)
- TF-IDF 벡터화
- 워드 클라우드 시각화
- 문서 유사도 분석